In [29]:
!pip install mlflow
!pip install dagshub

In [30]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import joblib

In [31]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [32]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def remove_stop_words(text):
    stop_words = set(stopwords.words("english"))
    return " ".join([word for word in text.split() if word not in stop_words])

def removing_numbers(text):
    return re.sub(r'\d+', '', text)

def lower_case(text):
    return text.lower()

def removing_punctuations(text):
    text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)

def normalize_text(df):
    try:
        df['content'] = (
            df['content']
            .astype(str)
            .apply(lower_case)
            .apply(remove_stop_words)
            .apply(removing_numbers)
            .apply(removing_punctuations)
            .apply(removing_urls)
            .apply(lemmatization)
        )
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [33]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
df = normalize_text(df)
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [35]:
df['sentiment'].value_counts()

,count
sentiment,
neutral,8638
worry,8459
happiness,5209
sadness,5165
love,3842
surprise,2187
fun,1776
relief,1526
hate,1323


In [36]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]


In [37]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()

/tmp/ipython-input-1089524538.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want married ...
8,0,charviray charlene love miss
9,0,kelcouch sorry least friday


In [38]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
import dagshub

dagshub.init(repo_owner='Aryanupadhyay23', repo_name='mlops-mini-project', mlflow=True)
mlflow.set_tracking_uri('https://dagshub.com/Aryanupadhyay23/mlops-mini-project.mlflow')

mlflow.set_experiment("Logistic Regression Baseline model")

Initialized MLflow to track repo "Aryanupadhyay23/mlops-mini-project"

Repository Aryanupadhyay23/mlops-mini-project initialized!

<Experiment: artifact_location='mlflow-artifacts:/4605b4344db44944957d2251eb88da68', creation_time=1762430883902, experiment_id='1', last_update_time=1762430883902, lifecycle_stage='active', name='Logistic Regression Baseline model', tags={}>

In [41]:
 with mlflow.start_run():
    # Log preprocessing parameters
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 1000)
    mlflow.log_param("test_size", 0.2)

    # Model building and training
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")

    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log evaluation metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # --- Log model ---
    import joblib
    joblib.dump(model, "exp1-base-model.pkl")
    mlflow.log_artifact("exp1-base-model.pkl")

    # Print results for verification
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")


Accuracy: 0.776867469879518
Precision: 0.7690058479532164
Recall: 0.7773399014778325
F1 Score: 0.7731504164625184
🏃 View run kindly-kit-528 at: https://dagshub.com/Aryanupadhyay23/mlops-mini-project.mlflow/#/experiments/1/runs/e6863634acd5490bb47f84cdee821399
🧪 View experiment at: https://dagshub.com/Aryanupadhyay23/mlops-mini-project.mlflow/#/experiments/1
